In [1]:
import numpy as np

In [2]:
def generateBitstrings(n):
    bitstrings = []
    for i in range(2**n):
        bitstrings.append(f'{i:0{n}b}')
    return bitstrings

print(generateBitstrings(3))

['000', '001', '010', '011', '100', '101', '110', '111']


In [3]:
def addBitstrings(x, y):
    z = []
    for i, bit in enumerate(x):
        z.append(str((int(bit) + int(y[i])) % 2))
    return ''.join(z)

print(addBitstrings('0011', '0101'))

0110


In [4]:
def generateF(bitstrings, s):
    f = {}
    for x in bitstrings:
        if x not in f:
            y = x if x[-1] == '0' else addBitstrings(x, s)
            f[y] = y
            f[addBitstrings(y, s)] = y
    return f

print(generateF(generateBitstrings(3), '111'))

{'000': '000', '111': '000', '110': '110', '001': '110', '010': '010', '101': '010', '100': '100', '011': '100'}


In [5]:
def hammingDistance(x, y):
    dist = 0
    for i, bit in enumerate(x):
        if bit != y[i]:
            dist += 1
    return dist

print(hammingDistance('0011', '0101'))

2


In [6]:
def bitstringToVector(x):
    v = np.zeros(2**len(x))
    v[int(x, 2)] = 1
    return v

print(bitstringToVector('101'))

[0. 0. 0. 0. 0. 1. 0. 0.]


In [7]:
# def hamiltonian(n, s=None, verbose=False):
#     outerBitstrings = generateBitstrings(n)
#     innerBitstrings = generateBitstrings(n-1)
#     if s == None:
#         s = outerBitstrings[-1]
#     f = generateF(outerBitstrings, s)
#     outerSum = np.zeros((2**(2*n-1), 2**(2*n-1)))
#     for x in outerBitstrings:
#         v = bitstringToVector(x)
#         innerSum = np.zeros((2**(n-1), 2**(n-1)))
#         for y in innerBitstrings:
#             w = bitstringToVector(y)
#             innerSum += hammingDistance(y, f[x]) * np.outer(w, w)
#         outerSum += np.kron(np.outer(v, v), innerSum)
#     if verbose:
#         print('Outer bitstrings:', outerBitstrings)
#         print('Inner bitstrings:', innerBitstrings)
#         print('Function f(x):   ', f)
#     return outerSum

def hamiltonian(bitstrings, f):
    n = len(bitstrings[0])
    outerSum = np.zeros((2**(2*n), 2**(2*n)))
    for x in bitstrings:
        v = bitstringToVector(x)
        innerSum = np.zeros((2**n, 2**n))
        for y in bitstrings:
            w = bitstringToVector(y)
            innerSum += hammingDistance(y, f[x]) * np.outer(w, w)
        outerSum += np.kron(np.outer(v, v), innerSum)
    return outerSum

bitstrings = generateBitstrings(2)
print(bitstrings)
f = generateF(bitstrings, bitstrings[-1])
print(f)
qubo = hamiltonian(bitstrings, f)
# qubo = hamiltonian(2, verbose=True)
qubo

['00', '01', '10', '11']
{'00': '00', '11': '00', '10': '10', '01': '10'}


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [8]:
def solveQubo(q, n):
    mins = set()
    minExpectation = np.inf
    for i in range(2**(2**(2*n))):
        x = f'{i:0{2**(2*n)}b}'
        v = np.array(list(map(int, list(x))))
        expectation = v.T @ q @ v
        # if expectation <= 0:
        #     print(x, expectation)
        if expectation < minExpectation:
            minExpectation = expectation
            mins = set()
            mins.add(x)
        if expectation == minExpectation:
            mins.add(x)
    return mins, minExpectation

def interpretResults(results, n):
    candidates = set()
    decodedResults = set()
    for result in results:
        indices = list(find_all(result, '1'))
        for index in indices:
            if index not in candidates:
                print(f'{index:0{2*n}b}')
                candidates.add(index)
            decodedResults.add(f'{index:0{2*n}b}'[n:])
    return decodedResults

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)

In [9]:
results, expectation = solveQubo(qubo, 2)
results

{'0000000000000000',
 '0000000000001000',
 '0000000000100000',
 '0000000000101000',
 '0000001000000000',
 '0000001000001000',
 '0000001000100000',
 '0000001000101000',
 '1000000000000000',
 '1000000000001000',
 '1000000000100000',
 '1000000000101000',
 '1000001000000000',
 '1000001000001000',
 '1000001000100000',
 '1000001000101000'}

In [10]:
interpretResults(results, 2)

1010
1100
0000
0110


{'00', '10'}